In [1]:
import os
from IPython.display import clear_output
from google.colab import output
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
#Check video module 
#USE ONLY Tesla P100-PCIE
!nvidia-smi

Sun May 24 19:38:45 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.82       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   42C    P8     7W /  75W |      0MiB /  7611MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [3]:
os.chdir('/content/drive/My Drive')
project_folder = 'BACR_Project'
Controller_folder = 'Controller'
if not os.path.exists(os.path.join(project_folder, Controller_folder)):
  os.mkdir(project_folder)
os.chdir(project_folder)
if not os.path.exists(Controller_folder):
  os.mkdir(Controller_folder)  
os.chdir(Controller_folder)
!pwd  

/content/drive/My Drive/BACR_Project/Controller


# Install dependencies

In [0]:
!apt-get update && apt-get upgrade -y
!apt-get install -y software-properties-common
!apt-get update && apt-get install -y freeglut3-dev git
!apt-get update --fix-missing && apt-get install -y \
      python3-numpy python3-matplotlib python3-dev \
      python3-opengl python3-pip \
      cmake zlib1g-dev libjpeg-dev xvfb libpq-dev \
      xorg-dev libboost-all-dev libsdl2-dev swig \
      git wget openjdk-8-jdk ffmpeg unzip\
    && apt-get clean && rm -rf /var/cache/apt/archives/* /var/lib/apt/lists/*
!pip3 install --upgrade pip
!pip3 install h5py future pyvirtualdisplay 'gym[atari]' 'gym[box2d]' 'gym[classic_control]' opencv-python
!pip3 install torch torchvision tensorflow-gpu==1.8 pandas filelock
!pip3 install pybullet==2.5.0
!sed -i 's/robot_bases/pybullet_envs.robot_bases/' /usr/local/lib/python3.5/dist-packages/pybullet_envs/robot_locomotors.py
!apt-get update && apt-get install libgsl0-dev -y
!pip3 install pyglet Cython
!git clone https://github.com/snolfi/evorobotpy.git
!cd evorobotpy/lib/ \
    && python3 setupevonet.py build_ext --inplace \
    && cp net*.so ../bin
!cd evorobotpy/lib/ \
    && python3 setupErDiscrim.py build_ext --inplace \
    && cp ErDiscrim*.so ../bin
!сd evorobotpy/lib/ \
    && python3 setupErPredprey.py build_ext --inplace \
    && cp ErPredprey*.so ../bin   
!cd evorobotpy/lib/ \
    && python3 setupErDpole.py build_ext --inplace \
    && cp ErDpole*.so ../bin
!pip3 install notebook
!git clone --depth 1 https://github.com/openai/baselines.git \
    && sed --in-place 's/mujoco,//' baselines/setup.py \
    && cd baselines && pip3 install -e . \
    && pip3 install mpi4py cloudpickle             
output.clear()



# Clone our repository 

In [5]:
os.chdir('/content/drive/My Drive/BACR_Project/Controller')
!git clone https://github.com/Terminateit/BACR_MIT_CAR
% cd BACR_MIT_CAR/
!git checkout controller

Cloning into 'BACR_MIT_CAR'...
remote: Enumerating objects: 3814, done.
remote: Counting objects: 100% (3814/3814), done.
remote: Compressing objects: 100% (425/425), done.
remote: Total 3814 (delta 3488), reused 3698 (delta 3382), pack-reused 0
Receiving objects: 100% (3814/3814), 46.26 MiB | 11.21 MiB/s, done.
Resolving deltas: 100% (3488/3488), done.
Checking out files: 100% (151/151), done.
/content/drive/My Drive/BACR_Project/Controller/BACR_MIT_CAR
Branch 'controller_dev' set up to track remote branch 'controller_dev' from 'origin'.
Switched to a new branch 'controller_dev'


# Substitute files

In [6]:
% cd controller 
!cp controller.py ../../evorobotpy/bin/
!cp es.py ../../evorobotpy/bin/
!cp es_car.py ../../evorobotpy/bin/
!cp policy_car.py ../../evorobotpy/bin/
!cp mdrnn.py ../../evorobotpy/bin/
!cp vae.py ../../evorobotpy/bin/

#folder 
! cp -r models_dir ../../evorobotpy/bin/

#ini file
!cp racecar_env2.ini ../racecar/

/content/drive/My Drive/BACR_Project/Controller/BACR_MIT_CAR/controller


# Build the env

In [7]:
% cd /content/drive/My Drive/BACR_Project/Controller/BACR_MIT_CAR/racecar
! pip3 install -e. 

/content/drive/My Drive/BACR_Project/Controller/BACR_MIT_CAR/racecar
Obtaining file:///content/drive/My%20Drive/BACR_Project/Controller/BACR_MIT_CAR/racecar
  Running setup.py develop for racecar
  Attempting uninstall: cloudpickle
    Found existing installation: cloudpickle 1.3.0
    Uninstalling cloudpickle-1.3.0:
      Successfully uninstalled cloudpickle-1.3.0


# Build NET

In [8]:
% cd /content/drive/My Drive/BACR_Project/Controller/evorobotpy/lib
! python3 setupevonet.py build_ext --inplace
!cp net*.so ../bin # or cp net*.dll ../bin

/content/drive/My Drive/BACR_Project/Controller/evorobotpy/lib
running build_ext
skipping 'net.cpp' Cython extension (up-to-date)


# Launch es_car.py

In [0]:
% cd /content/drive/My Drive/BACR_Project/Controller/BACR_MIT_CAR/racecar
! python3 ../../evorobotpy/bin/es_car.py -f racecar_env2.ini -s 15

/content/drive/My Drive/BACR_Project/Controller/BACR_MIT_CAR/racecar
Environment racecar-v2 nreplications 1 maxmsteps 50m 
pybullet build time: May 24 2020 19:42:24
/usr/local/lib/python3.6/dist-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float64
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))
Car Gym policy
Evaluation: Episodes 1 Test Episodes 1 MaxSteps 100000
Network 288->100->3 recurrent with bias tanh output:linear init:xavier input-normalization output-diagonal-gaussian 
Run Evolve: Environment racecar-v2 Seed 15 Nreplications 1
Salimans: seed 15 maxmsteps 50 batchSize 10 stepsize 0.010000 noiseStdDev 0.020000 wdecay 0 sameEnvCond 0 nparams 39215
